In [4]:
import itertools
from utils import *
import copy

# Parsing

In [5]:
with open("data/input12.txt") as f:
    data = f.read().rstrip("\n")

ls = [l for l in data.split("\n")]
inps = [[c for c in l.split(" ")[0]] for l in ls]
nbrs = [ints(l.split(" ")[1]) for l in ls]

# Part 1

In [6]:
def conforms(arr, nbr):
    prev = "."
    seglen = 0
    seglens = []
    for ins in arr:
        if ins=="#":
            if prev==".":
                seglen = 1
            else:
                seglen += 1
        elif prev=="#":
            seglens.append(seglen)
        prev = ins
    if prev=="#":
        seglens.append(seglen)
    if len(seglens)!=len(nbr):
        return False
    for truelen, testlen in zip(nbr, seglens):
        if truelen!=testlen:
            return False
    return True

accum = 0
for inp, nbr in zip(inps, nbrs):
    nn = len(inp)
    qi = [i for i in range(len(inp)) if inp[i]=="?"]
    arr = []
    for i in range(nn):
        if inp[i]=="#":
            arr.append("#")
        else:
            arr.append(".")
    ncombs = 0
    for comb in itertools.product([True, False], repeat=len(qi)):
        test = copy.copy(arr)
        for i, j in enumerate(qi):
            if comb[i]:
                test[j] = "#"
        if conforms(test, nbr):
            ncombs += 1
    accum += ncombs
print(accum)

7916


# Part 2

In [7]:
inps = [l.split(" ")[0] for l in ls]
nbrs = [ints(l.split(" ")[1]) for l in ls]
for i in range(len(inps)):
    tempinp = copy.copy(inps[i])
    tempnbr = copy.copy(nbrs[i])
    for _ in range(4):
        inps[i] += "?" + tempinp
        nbrs[i] += tempnbr

def recurse_line(inp, nbr, curnbr):
    newpos = inp.find("?")
    if newpos<0:
        if conforms(inp, nbr):
            return 1
        else:
            return 0

    newnbr, accum = count_up(inp[:newpos],nbr)
    if curnbr>=len(nbr) and accum>0:
        return 0

    comb_computed = (newpos, newnbr, accum)
    if comb_computed in lookup_table:
        return lookup_table[comb_computed]

    result = 0
    for c in "#.":
        inp1 = inp[:newpos] + c + inp[newpos+1:]
        newcurnbr,_ = count_up(inp1[:newpos+1],nbr)
        if newcurnbr>=0:
            result += recurse_line(inp1, nbr, newcurnbr)

    lookup_table[comb_computed] = result
    return result

def count_up(line, nbr):
    prev = "."
    curnbrout = 0
    accum = 0
    for c in line:
        if c=="." and prev=="#":
            if curnbrout>=len(nbr) or accum!=nbr[curnbrout]:
                return -1, -1
            curnbrout += 1
            accum = 0
        elif c=="#":
            if curnbrout>=len(nbr) or (prev=="#" and accum+1>nbr[curnbrout]):
                return -1, -1
            accum += 1
        prev=c
    if curnbrout<len(nbr) and accum>nbr[curnbrout]:
        return -1,-1
    return curnbrout,accum

lookup_table = {} # Use a lookup table to remember results from previous checks of the same combinations
totcomb = 0
for inp, nbr in zip(inps, nbrs):
    lookup_table.clear()
    combs = recurse_line("." + inp + ".", nbr, 0)
    totcomb += combs
totcomb

37366887898686